# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115604e+02     1.590712e+00
 * time: 0.1177670955657959
     1     1.004519e+01     9.851803e-01
 * time: 0.9153361320495605
     2    -1.211022e+01     1.070335e+00
 * time: 1.0524959564208984
     3    -3.412933e+01     8.075901e-01
 * time: 1.2004051208496094
     4    -4.738603e+01     5.852293e-01
 * time: 1.3405251502990723
     5    -5.700948e+01     2.084137e-01
 * time: 1.4791419506072998
     6    -5.983078e+01     1.379781e-01
 * time: 1.582827091217041
     7    -6.092426e+01     4.888245e-02
 * time: 1.693068027496338
     8    -6.138168e+01     3.718360e-02
 * time: 1.7973310947418213
     9    -6.165040e+01     3.506501e-02
 * time: 1.9083549976348877
    10    -6.182425e+01     3.829428e-02
 * time: 2.0263309478759766
    11    -6.196948e+01     3.406259e-02
 * time: 2.130929946899414
    12    -6.207026e+01     2.609279e-02
 * time: 2.2417850494384766
    13    -6.211888e+01     1.581326e-02
 * time: 2.3533310890197

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557658
    AtomicNonlocal      14.8522630
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336820

    total               -62.261666452924
